In [18]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from awsglue.dynamicframe import DynamicFrame
import re

glueContext = GlueContext(SparkContext.getOrCreate())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
job = Job(glueContext)
job.init('from_glue_table_to_snowflake')

dyf_installed_base = glueContext.create_dynamic_frame.from_catalog(database='hdm_rc_landing_zone', 
                                                                   table_name='rcl_installed_base')
df_installed_base = dyf_installed_base.toDF()
for column in df_installed_base.columns:
    sf_column = re.sub(r'[^a-zA-Z0-9_]', '_', column)
    sf_column = re.sub(r'^([^a-zA-Z])', r'_\1', sf_column)
    sf_column = re.sub(r'_{2,}', '_', sf_column)
    sf_column = re.sub(r'_+$', '', sf_column)
    if sf_column != column:
        df_installed_base = df_installed_base.withColumnRenamed(column, sf_column)
dyf_installed_base = DynamicFrame.fromDF(df_installed_base, glueContext, 'df_installed_base')

glueContext.write_dynamic_frame.from_options(
    frame=dyf_installed_base,
    connection_type='custom.jdbc',
    connection_options={
        'tableName': 'rcl_installed_base',
        'dbTable': 'rcl_installed_base',
        'connectionName': 'snowflake-glue-jdbc-connection',
    },
    transformation_ctx="df_installed_base",
)

job.commit()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
An error occurred while calling o981.pyWriteDynamicFrame.
: java.sql.SQLException: No suitable driver
	at java.sql.DriverManager.getDriver(DriverManager.java:315)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions$$anonfun$6.apply(JDBCOptions.scala:105)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions$$anonfun$6.apply(JDBCOptions.scala:105)
	at scala.Option.getOrElse(Option.scala:121)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:104)
	at org.apache.spark.sql.jdbc.glue.GlueJDBCOptions.<init>(GlueJDBCOptions.scala:14)
	at org.apache.spark.sql.jdbc.glue.GlueJDBCOptions.<init>(GlueJDBCOptions.scala:17)
	at com.amazonaws.services.glue.marketplace.partner.PartnerJDBCRecordWriterFactory.<init>(PartnerJDBCDataSink.scala:78)
	at com.amazonaws.services.glue.marketplace.partner.PartnerJDBCDataSink.createWriterFactory(PartnerJDBCDataSink.scala:32)
	at com.amazonaws.services.glue.marketplace.partner.Pa